From https://www.kaggle.com/chauhuynh/my-first-kernel-3-699

https://www.kaggle.com/saurabh502/why-no-simple-blend-plb-3-687  3.687

In [46]:
import numpy as np
import pandas as pd
import datetime
import gc
import time
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, RepeatedKFold, KFold
from sklearn.metrics import mean_squared_error, roc_auc_score, log_loss
from sklearn.impute  import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge, LinearRegression

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE_TSG = 2019
np.random.seed(RANDOM_STATE_TSG)

file_path = 'F:/github_me_repos/Kaggle_code/elo-merchant-category-recommendation/'
print('finished!')

hello world!


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

print('finished!')

finished!


In [3]:
df_train = pd.read_csv('./train_first_kernel.csv')
df_test = pd.read_csv('./test_first_kernel.csv')

print('finished!')

finished!


In [4]:
df_train.shape, df_test.shape, df_train['target'].mean(), df_train['target'].std()
# df_train['hist_purchase_date_max'].head(20)
# df_train.dtypes

((201917, 107), (123623, 105), -0.3936423574869955, 3.850565426186068)

In [5]:
 df_train[df_train['target']>=-30.]['target'].mean(), df_train[df_train['target']>=-30.]['target'].std()

(-0.030877491846677468, 1.717788982308337)

In [6]:
 df_train[df_train['target']>=-30.]['target'].min(),  df_train[df_train['target']>=-30.]['target'].max(), df_train['target'].min()

(-17.61, 17.97, -33.22)

In [7]:
# for fea_name in ['feature_1', 'feature_2', 'feature_3', 'feature1_and_feature2', 
#                  'feature1_and_feature3', 'feature3_and_feature2', 'feature1_and_feature2_and_feature3']:
#     if fea_name in df_test.columns.values:
#         print('fea_name ', fea_name, 'in columns')

In [8]:
# print('prev columns df_train.columns ', df_train.columns.values)

In [9]:
def add_features_interaction(df_train, df_test):
    for df in (df_train, df_test):
        df['feature1_and_feature2'] = df['feature_1']*df['feature_2']*100.0
        df['feature1_and_feature3'] = df['feature_1']*df['feature_3']*100.0
        df['feature3_and_feature2'] = df['feature_3']*df['feature_2']*100.0
        df['feature1_and_feature2_and_feature3'] = df['feature_1']*df['feature_2']*df['feature_3']*10000.0
    print('processing_features')
    return 

# add_features_interaction(df_train, df_test)
# reduce_mem_usage(df_train)
# reduce_mem_usage(df_test)



# df_train = pd.get_dummies(df_train, dummy_na=True, 
#                           columns=['feature_1', 'feature_2', 'feature_3',
#                                    'feature1_and_feature2', 'feature1_and_feature3',
#                                    'feature3_and_feature2', 'feature1_and_feature2_and_feature3'])
# df_test = pd.get_dummies(df_test, dummy_na=True, 
#                           columns=['feature_1', 'feature_2', 'feature_3',
#                                    'feature1_and_feature2', 'feature1_and_feature3',
#                                    'feature3_and_feature2', 'feature1_and_feature2_and_feature3'])

df_train = pd.get_dummies(df_train, dummy_na=True, 
                          columns=['feature_1', 'feature_2', 'feature_3'])
df_test = pd.get_dummies(df_test, dummy_na=True, 
                          columns=['feature_1', 'feature_2', 'feature_3'])

print('after columns df_train.columns ', df_train.columns.values)
reduce_mem_usage(df_train)
reduce_mem_usage(df_test)

after columns df_train.columns  ['first_active_month' 'card_id' 'target' 'hist_month_nunique'
 'hist_hour_nunique' 'hist_weekofyear_nunique' 'hist_dayofweek_nunique'
 'hist_year_nunique' 'hist_subsector_id_nunique'
 'hist_merchant_id_nunique' 'hist_merchant_category_id_nunique'
 'hist_city_id_nunique' 'hist_state_id_nunique' 'hist_purchase_amount_sum'
 'hist_purchase_amount_max' 'hist_purchase_amount_min'
 'hist_purchase_amount_mean' 'hist_purchase_amount_var'
 'hist_installments_sum' 'hist_installments_max' 'hist_installments_min'
 'hist_installments_mean' 'hist_installments_var' 'hist_purchase_date_max'
 'hist_purchase_date_min' 'hist_month_lag_max' 'hist_month_lag_min'
 'hist_month_lag_mean' 'hist_month_lag_var' 'hist_month_diff_mean'
 'hist_authorized_flag_sum' 'hist_authorized_flag_mean' 'hist_weekend_sum'
 'hist_weekend_mean' 'hist_category_1_sum' 'hist_category_1_mean'
 'hist_card_id_size' 'hist_category_2_mean_mean'
 'hist_category_3_mean_mean' 'hist_purchase_date_diff'
 'hist_

,first_active_month,card_id,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,...,feature_1_0.010711591651998994,feature_1_0.013144615384615385,feature_1_nan,feature_2_0.008752121220219405,feature_2_0.011384773985343223,feature_2_0.014166402368115024,feature_2_nan,feature_3_0.01028283496163887,feature_3_0.011427520622077443,feature_3_nan
0,2017-04-01,C_ID_0ab67a22ab,9,17,24,7,1,12,24,16,...,0,0,0,0,0,1,0,0,1,0
1,2017-01-01,C_ID_130fd0cbdd,5,18,20,7,2,12,27,16,...,0,0,0,0,0,1,0,1,0,0
2,2017-08-01,C_ID_b709037bc5,6,7,7,4,2,6,9,8,...,0,1,0,0,1,0,0,0,1,0
3,2017-12-01,C_ID_d27d835a9f,3,9,6,7,2,11,23,18,...,0,0,0,0,1,0,0,1,0,0
4,2015-12-01,C_ID_2b5e3df5c2,12,21,34,7,2,15,47,31,...,0,1,0,0,1,0,0,0,1,0
5,2017-07-01,C_ID_5814b4f13c,7,11,9,5,2,4,11,4,...,0,1,0,0,1,0,0,0,1,0
6,2017-10-01,C_ID_a1b3c75277,5,16,13,7,2,14,25,17,...,0,0,0,1,0,0,0,1,0,0
7,2017-02-01,C_ID_f7cada36d3,8,11,12,7,2,4,6,4,...,0,0,0,0,1,0,0,0,1,0
8,2017-07-01,C_ID_9d2bc8dfc4,8,14,25,7,2,12,19,14,...,1,0,0,0,0,1,0,1,0,0
9,2016-03-01,C_ID_6d8dba8475,7,18,15,7,1,10,32,16,...,0,1,0,0,1,0,0,0,1,0


In [10]:
# df_train['feature_2'].value_counts()

In [11]:
# df_test['feature_3'].value_counts()

In [12]:
# df_train_no_outliers = df_train[df_train['outliers']==0]
# df_train['outliers'].value_counts()

In [13]:
# https://www.kaggle.com/adarshchavakula/out-of-fold-oof-model-cross-validation

# # Read Data
# print("Reading Dataset...")
# train = pd.read_csv("../input/train.csv")
# target = np.array(train["target"])
# target_log = np.log1p(target) # Log transform target as the evaluation metric uses it
# xtrain = np.array(train.iloc[:,2:])
# print("Shape of training data: {}".format(np.shape(xtrain)))

# # Define Model 
# xgb_model = XGBRegressor(max_depth=6, learning_rate=0.1, n_estimators=70,
#                          min_child_weight=100, subsample=1.0, 
#                          colsample_bytree=0.8, colsample_bylevel=0.8,
#                          random_state=42, n_jobs=4)

# # Make OOF predictions using 5 folds
# print("Cross Validating...")
# oof_preds_log = cross_val_predict(xgb_model, xtrain, target_log, cv=5, 
#                                   n_jobs=1, method="predict")
                                  
# # Calculate RMSLE (RMSE of Log(1+y))
# cv_rmsle = np.sqrt(mean_squared_error(target_log, oof_preds_log))
# print("\nOOF RMSLE Score: {:.4f}".format(cv_rmsle))

In [14]:
def compute_val_rmse(df_train, trn_idx, val_idx, preds_classifier, preds_regressor_full, preds_regressor_partial, partial_ratio=0.10):
    print('in compute_val_rmse')
#     partial_ratio = 0.10
    df_is_outlier = pd.DataFrame({"card_id": df_train["card_id"].iloc[val_idx].values})
    df_is_outlier['outlier'] =  preds_classifier
    df_is_outlier['target'] =  preds_regressor_full
    outlier_num = int(partial_ratio*len(val_idx))  # outlier 个数
   
    df_is_outlier.sort_values(by='outlier',ascending = False, inplace=True)
    df_is_outlier = df_is_outlier[:outlier_num]
    df_is_outlier = df_is_outlier[['card_id', 'target']]
    df_is_outlier.set_index('card_id', inplace=True)
    
    final_outcome_df = pd.DataFrame({"card_id": df_train["card_id"].iloc[val_idx].values})
    final_outcome_df['target'] = preds_regressor_partial
    
    final_outcome_df.set_index('card_id', inplace=True)
    final_outcome_df.loc[df_is_outlier.index, 'target'] = df_is_outlier['target']
    target_new = final_outcome_df.loc[df_train['card_id'].iloc[val_idx], 'target'].values
    
    rmse_val = np.sqrt(mean_squared_error(df_train['target'].iloc[val_idx], target_new))    
    return target_new, rmse_val

def compute_rmse_check_ratio(df_train, trn_idx_list, val_idx_list, preds_classifier_list, 
                             preds_regressor_full_list, preds_regressor_partial_list, 
                             partial_ratio=0.10):
    print('in compute_rmse_check_ratio partial_ratio: ', partial_ratio)
    
    oof_regressor_merged = np.zeros(len(df_train))
    for trn_idx, val_idx, preds_classifier, preds_regressor_full, preds_regressor_partial in zip(
        trn_idx_list, val_idx_list, preds_classifier_list, preds_regressor_full_list, 
        preds_regressor_partial_list):
        oof_regressor_merged[val_idx], rmse = compute_val_rmse(df_train, trn_idx, val_idx, 
                     preds_classifier, preds_regressor_full, preds_regressor_partial, partial_ratio)
        
#         print('sub rmse is ', rmse)
    total_rmse = np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_merged))
    print('total_rmse is ', total_rmse)
    return total_rmse


def get_final_results(df_test, preds_classifier, preds_regressor_full, preds_regressor_partial, partial_ratio = 0.10):
    print('in get_final_results')
    df_is_outlier = pd.DataFrame({"card_id": df_test["card_id"].values})
    df_is_outlier['outlier'] =  preds_classifier
    df_is_outlier['target'] =  preds_regressor_full
    outlier_num = int(partial_ratio*len(df_test))  # outlier 个数
   
    df_is_outlier.sort_values(by='outlier',ascending = False, inplace=True)
    df_is_outlier = df_is_outlier[:outlier_num]
    df_is_outlier = df_is_outlier[['card_id', 'target']]
    df_is_outlier.set_index('card_id', inplace=True)
    
    final_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
    final_outcome_df['target'] = preds_regressor_partial
    
    final_outcome_df.set_index('card_id', inplace=True)
    final_outcome_df.loc[df_is_outlier.index, 'target'] = df_is_outlier['target']
    target_new = final_outcome_df.loc[df_test['card_id'].values, 'target'].values
    
    return target_new

print('finished!')

finished!


In [16]:
RANDOM_STATE_TSG = 10007
# target = df_train['outliers']

df_train.drop(columns=['is_outlier_prob'], inplace=True, errors='ignore')
df_test.drop(columns=['is_outlier_prob'], inplace=True, errors='ignore')

features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers', 'target']]
features_old = features
features = list(df_train[features].select_dtypes(include=['number', 'bool']).columns.values)  # 去除非数值、非布尔类型数据
# features = [fea for fea in features if fea not in ['feature1_and_feature2', 'feature1_and_feature3', 
#                                                    'feature3_and_feature2', 'feature1_and_feature2_feature3']]

# categorical_feats = [c for c in features if 'feature_' in c]
categorical_feats = []
print(len(features), len(categorical_feats))

113 0


In [16]:
# # 增加一列 特征 'is_outlier_prob'
# param_classifier = {
#          'num_leaves': 31,
#          'min_data_in_leaf': 30, 
#          'objective':'binary',
# #          'max_depth': 6,
#          'max_depth': -1,
#          'learning_rate': 0.01,
# #          "boosting": "rf",
#          "boosting": "gbdt",
#          "feature_fraction": 0.9,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.9 ,
#          "bagging_seed": 11,
#          "metric": 'binary_logloss',
#          "lambda_l1": 0.1,
#          "verbosity": -1,
# #          "random_state": 2333,
#          'random_state': RANDOM_STATE_TSG
# }  # final auc is  0.9030288798685597


# folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE_TSG)
# # folds = KFold(n_splits=5, shuffle=True, random_state=15)

# oof_classifier = np.zeros(len(df_train))
# predictions_classifier = np.zeros(len(df_test))

# start = time.time()

# for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
#     print("fold n°{}".format(fold_))
    
#     # 训练一个分类器 决定是否为 outlier
# #     trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['outliers'].iloc[trn_idx], categorical_feature=categorical_feats)
# #     val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['outliers'].iloc[val_idx], categorical_feature=categorical_feats)
#     trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['outliers'].iloc[trn_idx])
#     val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['outliers'].iloc[val_idx])

#     num_round = 10000
#     print('start classifier training...')
#     clf = lgb.train(param_classifier, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
#     oof_classifier[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

#     auc = roc_auc_score(df_train['outliers'].iloc[val_idx], oof_classifier[val_idx])
#     print('current auc is ', auc)
    
#     predictions_classifier += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
# df_train['is_outlier_prob'] = oof_classifier
# df_test['is_outlier_prob'] = predictions_classifier

# features_regressor = features + ['is_outlier_prob']

# print('final auc is ', roc_auc_score(df_train['outliers'].values, oof_classifier))

# ###########################################################################################

# print('start for training for target ')

In [17]:
param_regressor_1 = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.007,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         'random_state': 2019
}


folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)

oof_regressor_full_1 = np.zeros(len(df_train))
predictions_regressor_full_1 = np.zeros(len(df_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['target'].iloc[trn_idx], 
                           categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['target'].iloc[val_idx], 
                           categorical_feature=categorical_feats)

    num_round = 10000
    print('start regressor training...')
    clf = lgb.train(param_regressor_1, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_regressor_full_1[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)    
    predictions_regressor_full_1 += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    rmse_val = np.sqrt(mean_squared_error(df_train.iloc[val_idx]['target'].values, 
                                          oof_regressor_full_1[val_idx]))
    print('current rmse val is ', rmse_val)
    
print('final rmse val 1 is ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_full_1)))

# final_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
# final_outcome_df['target'] = get_final_results(df_test, predictions_classifier, 
#                                                predictions_regressor_full, predictions_regressor_partial,
#                                                partial_ratio=0.25)

# final_outcome_df.to_csv(file_path + '/submission/submission_first_kernel_tsg.csv', index=False)

fold n°0
start regressor training...
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.70103	valid_1's rmse: 3.74246
[200]	training's rmse: 3.62965	valid_1's rmse: 3.70186
[300]	training's rmse: 3.58365	valid_1's rmse: 3.68404
[400]	training's rmse: 3.54982	valid_1's rmse: 3.6738
[500]	training's rmse: 3.52213	valid_1's rmse: 3.6668
[600]	training's rmse: 3.4991	valid_1's rmse: 3.66235
[700]	training's rmse: 3.47841	valid_1's rmse: 3.6589
[800]	training's rmse: 3.4609	valid_1's rmse: 3.65689
[900]	training's rmse: 3.444	valid_1's rmse: 3.65481
[1000]	training's rmse: 3.42899	valid_1's rmse: 3.65333
[1100]	training's rmse: 3.41498	valid_1's rmse: 3.65234
[1200]	training's rmse: 3.40122	valid_1's rmse: 3.6516
[1300]	training's rmse: 3.3881	valid_1's rmse: 3.65084
[1400]	training's rmse: 3.37532	valid_1's rmse: 3.65034
[1500]	training's rmse: 3.36334	valid_1's rmse: 3.64992
[1600]	training's rmse: 3.35152	valid_1's rmse: 3.64935
[1700]	training's rmse

In [18]:
param_regressor_2 = {
    'task': 'train',
    'boosting': 'goss',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.01,
    'subsample': 0.9855232997390695,
    'max_depth': -1,
    'top_rate': 0.9064148448434349,
    'num_leaves': 31,
    'min_child_weight': 41.9612869171337,
    'other_rate': 0.0721768246018207,
    'reg_alpha': 9.677537745007898,
    'colsample_bytree': 0.5665320670155495,
    'min_split_gain': 9.820197773625843,
    'reg_lambda': 0.1,
    'min_data_in_leaf': 32,
    'verbose': -1,
    'seed':int(2**7),
    'bagging_seed':2015,
    ''random_state': 2019'
    'drop_seed':int(2**8)
}

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_regressor_full_2 = np.zeros(len(df_train))
predictions_regressor_full_2 = np.zeros(len(df_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['target'].iloc[trn_idx], 
                           categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['target'].iloc[val_idx], 
                           categorical_feature=categorical_feats)

    num_round = 10000
    print('start regressor training...')
    clf = lgb.train(param_regressor_2, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_regressor_full_2[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)    
    predictions_regressor_full_2 += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    rmse_val = np.sqrt(mean_squared_error(df_train.iloc[val_idx]['target'].values, 
                                          oof_regressor_full_2[val_idx]))
    print('current rmse val is ', rmse_val)
    
print('final rmse val 1 is ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_full_2)))

# current rmse val is  3.6589503585888403
# final rmse val 1 is  3.6582454648471354

# final_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
# final_outcome_df['target'] = get_final_results(df_test, predictions_classifier, 
#                                                predictions_regressor_full, predictions_regressor_partial,
#                                                partial_ratio=0.25)

# final_outcome_df.to_csv(file_path + '/submission/submission_first_kernel_tsg.csv', index=False)

fold n°0
start regressor training...
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.67321	valid_1's rmse: 3.72437
[200]	training's rmse: 3.59849	valid_1's rmse: 3.69886
[300]	training's rmse: 3.55273	valid_1's rmse: 3.68968
[400]	training's rmse: 3.5189	valid_1's rmse: 3.68605
[500]	training's rmse: 3.49121	valid_1's rmse: 3.68305
[600]	training's rmse: 3.46878	valid_1's rmse: 3.68109
[700]	training's rmse: 3.44871	valid_1's rmse: 3.67974
[800]	training's rmse: 3.43031	valid_1's rmse: 3.6792
[900]	training's rmse: 3.41331	valid_1's rmse: 3.67898
[1000]	training's rmse: 3.3981	valid_1's rmse: 3.67874
[1100]	training's rmse: 3.38462	valid_1's rmse: 3.67837
[1200]	training's rmse: 3.37043	valid_1's rmse: 3.6782
[1300]	training's rmse: 3.35685	valid_1's rmse: 3.67826
[1400]	training's rmse: 3.34405	valid_1's rmse: 3.67859
Early stopping, best iteration is:
[1239]	training's rmse: 3.36539	valid_1's rmse: 3.67814
current rmse val is  3.678142174140379

In [27]:
param_regressor_3 = {
    'task': 'train',
    'boosting': 'dart',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.008,
    'subsample': 0.81,
    'max_depth': -1,
    'top_rate': 0.88,
    'num_leaves': 31,
    'min_child_weight': 41.9612869171337,
    'other_rate': 0.0721768246018207,
    'reg_alpha': 9.677537745007898,
    'colsample_bytree': 0.5665320670155495,
    'min_split_gain': 9.820197773625843,
    'reg_lambda': 0.1,
    'min_data_in_leaf': 32,
    'verbose': -1,
    'random_state': 1001,
    'bagging_seed':2015,
    'drop_seed':int(2**8)
}

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1001)

oof_regressor_full_3 = np.zeros(len(df_train))
predictions_regressor_full_3 = np.zeros(len(df_test))

rmse_val_lst = []
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['target'].iloc[trn_idx], 
                           categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['target'].iloc[val_idx], 
                           categorical_feature=categorical_feats)

    num_round = 10000
    print('start regressor training...')
    clf = lgb.train(param_regressor_3, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_regressor_full_3[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)    
    predictions_regressor_full_3 += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    rmse_val = np.sqrt(mean_squared_error(df_train.iloc[val_idx]['target'].values, 
                                          oof_regressor_full_3[val_idx]))
    print('current rmse val is ', rmse_val)
    rmse_val_lst.append(rmse_val)

print('rmse_val_lst is ', rmse_val_lst)
print('final rmse val 1 is ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_full_3)))

# final_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
# final_outcome_df['target'] = get_final_results(df_test, predictions_classifier, 
#                                                predictions_regressor_full, predictions_regressor_partial,
#                                                partial_ratio=0.25)

# final_outcome_df.to_csv(file_path + '/submission/submission_first_kernel_tsg.csv', index=False)

fold n°0
start regressor training...
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.79749	valid_1's rmse: 3.81503
[200]	training's rmse: 3.74801	valid_1's rmse: 3.77672
[300]	training's rmse: 3.70848	valid_1's rmse: 3.74863
[400]	training's rmse: 3.69344	valid_1's rmse: 3.73988
[500]	training's rmse: 3.67867	valid_1's rmse: 3.7314
[600]	training's rmse: 3.6674	valid_1's rmse: 3.72595
[700]	training's rmse: 3.65127	valid_1's rmse: 3.71755
[800]	training's rmse: 3.62834	valid_1's rmse: 3.70781
[900]	training's rmse: 3.60806	valid_1's rmse: 3.70035
[1000]	training's rmse: 3.5941	valid_1's rmse: 3.69615
[1100]	training's rmse: 3.58293	valid_1's rmse: 3.69412
[1200]	training's rmse: 3.56995	valid_1's rmse: 3.6911
[1300]	training's rmse: 3.56354	valid_1's rmse: 3.69021
[1400]	training's rmse: 3.55411	valid_1's rmse: 3.68876
[1500]	training's rmse: 3.54421	valid_1's rmse: 3.68717
[1600]	training's rmse: 3.53301	valid_1's rmse: 3.68514
[1700]	training's

Early stopping, best iteration is:
[3903]	training's rmse: 3.35482	valid_1's rmse: 3.66683
current rmse val is  3.667779346898787
fold n°3
start regressor training...
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.80118	valid_1's rmse: 3.80289
[200]	training's rmse: 3.75198	valid_1's rmse: 3.76265
[300]	training's rmse: 3.7129	valid_1's rmse: 3.73352
[400]	training's rmse: 3.69785	valid_1's rmse: 3.72402
[500]	training's rmse: 3.68342	valid_1's rmse: 3.71472
[600]	training's rmse: 3.6723	valid_1's rmse: 3.70879
[700]	training's rmse: 3.65612	valid_1's rmse: 3.69976
[800]	training's rmse: 3.63346	valid_1's rmse: 3.68897
[900]	training's rmse: 3.61322	valid_1's rmse: 3.68164
[1000]	training's rmse: 3.5991	valid_1's rmse: 3.67744
[1100]	training's rmse: 3.58849	valid_1's rmse: 3.67484
[1200]	training's rmse: 3.57579	valid_1's rmse: 3.6714
[1300]	training's rmse: 3.56969	valid_1's rmse: 3.67075
[1400]	training's rmse: 3.5606	valid_1's rmse: 3.6694
[

In [57]:
current_random_state = 1985

X_train = df_train[features].values
X_test = df_test[features].values
y_train = df_train['target'].values

print('before impute X_train.shape is', X_train.shape, 'X_test.shape is', X_test.shape,
      'y_train.shape is ', y_train.shape)
print('y_train is ', y_train.mean())

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(np.vstack([X_train, X_test]))
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)

print('after impute X_train.shape is', X_train.shape, 'X_test.shape is', X_test.shape,
      'y_train.shape is ', y_train.shape)

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(np.vstack([X_train, X_test]))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# eval_metric='MAE'
catboost_model = CatBoostRegressor(iterations=10000, depth=6, learning_rate=0.008, 
                                   loss_function='RMSE', colsample_bylevel=0.7,
                                   random_state=current_random_state)
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=current_random_state)

oof_regressor_full_catboost = np.zeros(len(df_train))
predictions_regressor_full_catboost = np.zeros(len(df_test))
current_rmse_lst = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    catboost_model.fit(X_train[trn_idx], y_train[trn_idx], cat_features=[], 
                       eval_set=(X_train[val_idx], y_train[val_idx]), verbose_eval=100, 
                       early_stopping_rounds = 300, plot=False)
    oof_regressor_full_catboost[val_idx] = catboost_model.predict(X_train[val_idx])
    predictions_regressor_full_catboost += catboost_model.predict(X_test) / folds.n_splits
    rmse_val = np.sqrt(mean_squared_error(y_train[val_idx], oof_regressor_full_catboost[val_idx]))
    print('current rmse val is ', rmse_val)
    current_rmse_lst.append(rmse_val)

print('current_random_state: ', current_random_state)
print('current_rmse_lst catboost: ', current_rmse_lst)
print('final rmse val catboost: ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_full_catboost)))

# catboost_model = CatBoostRegressor(iterations=3000, depth=6, learning_rate=0.02, 
#                                    loss_function='RMSE',
#                                    random_state=current_random_state)
# current_random_state:  3333
# current_rmse_lst catboost:  [3.687294346310899, 3.6664863311731986, 3.6604055690829367, 3.6036856139625932, 3.6676720872243487]
# final rmse val catboost:  3.657217692122858

# catboost_model = CatBoostRegressor(iterations=5000, depth=6, learning_rate=0.018, 
#                                    loss_function='RMSE', colsample_bylevel=0.71,
#                                    random_state=current_random_state)
# current_rmse_lst catboost:  [3.6876006652135334, 3.665112978950801, 3.6591028035806055, 3.6022628175887657, 3.6654096181893827]
# final rmse val catboost:  3.6560092344832857


# catboost_model = CatBoostRegressor(iterations=10000, depth=6, learning_rate=0.008, 
#                                    loss_function='RMSE', colsample_bylevel=0.7,
#                                    random_state=current_random_state)
# Shrink model to first 7254 iterations.
# current rmse val is  3.6645533289503707
# current_random_state:  1985
# current_rmse_lst catboost:  [3.685188117361316, 3.6638684959104277, 3.657555844209079, 3.602714012361015, 3.6645533289503707]
# final rmse val catboost:  3.6548807111138344

before impute X_train.shape is (201917, 113) X_test.shape is (123623, 113) y_train.shape is  (201917,)
y_train is  -0.3936
after impute X_train.shape is (201917, 113) X_test.shape is (123623, 113) y_train.shape is  (201917,)
fold n°0
0:	learn: 3.8669001	test: 3.8741905	best: 3.8741905 (0)	total: 70.1ms	remaining: 11m 40s
100:	learn: 3.7421013	test: 3.7666474	best: 3.7666474 (100)	total: 7.59s	remaining: 12m 24s
200:	learn: 3.7001508	test: 3.7334150	best: 3.7334150 (200)	total: 14.8s	remaining: 12m
300:	learn: 3.6809497	test: 3.7203220	best: 3.7203220 (300)	total: 21.9s	remaining: 11m 45s
400:	learn: 3.6699393	test: 3.7137696	best: 3.7137696 (400)	total: 28.9s	remaining: 11m 31s
500:	learn: 3.6613732	test: 3.7093595	best: 3.7093595 (500)	total: 36s	remaining: 11m 22s
600:	learn: 3.6536743	test: 3.7057712	best: 3.7057712 (600)	total: 42.8s	remaining: 11m 9s
700:	learn: 3.6477988	test: 3.7035512	best: 3.7035512 (700)	total: 50s	remaining: 11m 3s
800:	learn: 3.6422244	test: 3.7015242	best:

2600:	learn: 3.5722452	test: 3.6701568	best: 3.6701489 (2599)	total: 3m 6s	remaining: 8m 49s
2700:	learn: 3.5680391	test: 3.6698423	best: 3.6698423 (2700)	total: 3m 13s	remaining: 8m 43s
2800:	learn: 3.5642051	test: 3.6694642	best: 3.6694642 (2800)	total: 3m 20s	remaining: 8m 36s
2900:	learn: 3.5608103	test: 3.6690703	best: 3.6690547 (2897)	total: 3m 27s	remaining: 8m 28s
3000:	learn: 3.5573903	test: 3.6688825	best: 3.6688549 (2997)	total: 3m 34s	remaining: 8m 21s
3100:	learn: 3.5537858	test: 3.6686213	best: 3.6686157 (3099)	total: 3m 41s	remaining: 8m 13s
3200:	learn: 3.5500343	test: 3.6685757	best: 3.6685369 (3121)	total: 3m 48s	remaining: 8m 6s
3300:	learn: 3.5464943	test: 3.6683566	best: 3.6683216 (3283)	total: 3m 55s	remaining: 7m 58s
3400:	learn: 3.5431386	test: 3.6682324	best: 3.6681875 (3369)	total: 4m 2s	remaining: 7m 51s
3500:	learn: 3.5394774	test: 3.6680034	best: 3.6679783 (3485)	total: 4m 10s	remaining: 7m 44s
3600:	learn: 3.5359691	test: 3.6678810	best: 3.6678648 (3597)	t

4500:	learn: 3.5136234	test: 3.6601278	best: 3.6600993 (4497)	total: 5m 34s	remaining: 6m 48s
4600:	learn: 3.5103422	test: 3.6599375	best: 3.6599147 (4598)	total: 5m 41s	remaining: 6m 40s
4700:	learn: 3.5069567	test: 3.6598635	best: 3.6598176 (4652)	total: 5m 48s	remaining: 6m 32s
4800:	learn: 3.5039845	test: 3.6594833	best: 3.6594729 (4796)	total: 5m 55s	remaining: 6m 24s
4900:	learn: 3.5009585	test: 3.6593273	best: 3.6593213 (4888)	total: 6m 2s	remaining: 6m 16s
5000:	learn: 3.4981307	test: 3.6591773	best: 3.6591575 (4988)	total: 6m 8s	remaining: 6m 8s
5100:	learn: 3.4953045	test: 3.6589461	best: 3.6589083 (5092)	total: 6m 15s	remaining: 6m
5200:	learn: 3.4923860	test: 3.6587311	best: 3.6586667 (5190)	total: 6m 22s	remaining: 5m 52s
5300:	learn: 3.4894171	test: 3.6586745	best: 3.6586270 (5226)	total: 6m 29s	remaining: 5m 44s
5400:	learn: 3.4864260	test: 3.6586320	best: 3.6586270 (5226)	total: 6m 35s	remaining: 5m 37s
5500:	learn: 3.4837090	test: 3.6584238	best: 3.6584003 (5494)	total

6500:	learn: 3.4688817	test: 3.6068615	best: 3.6068532 (6498)	total: 7m 28s	remaining: 4m 1s
6600:	learn: 3.4659176	test: 3.6065656	best: 3.6065656 (6600)	total: 7m 35s	remaining: 3m 54s
6700:	learn: 3.4629547	test: 3.6064398	best: 3.6063635 (6659)	total: 7m 41s	remaining: 3m 47s
6800:	learn: 3.4600010	test: 3.6063900	best: 3.6063635 (6659)	total: 7m 48s	remaining: 3m 40s
6900:	learn: 3.4571681	test: 3.6061300	best: 3.6061170 (6892)	total: 7m 55s	remaining: 3m 33s
7000:	learn: 3.4546229	test: 3.6060231	best: 3.6060212 (6999)	total: 8m 2s	remaining: 3m 26s
7100:	learn: 3.4518082	test: 3.6058327	best: 3.6058238 (7087)	total: 8m 9s	remaining: 3m 19s
7200:	learn: 3.4490645	test: 3.6056361	best: 3.6056276 (7173)	total: 8m 15s	remaining: 3m 12s
7300:	learn: 3.4462822	test: 3.6055758	best: 3.6055546 (7295)	total: 8m 22s	remaining: 3m 5s
7400:	learn: 3.4437264	test: 3.6054028	best: 3.6054007 (7399)	total: 8m 29s	remaining: 2m 58s
7500:	learn: 3.4409391	test: 3.6053070	best: 3.6052916 (7497)	to

5100:	learn: 3.4922934	test: 3.6663576	best: 3.6663479 (5083)	total: 6m 14s	remaining: 5m 59s
5200:	learn: 3.4891997	test: 3.6663586	best: 3.6663177 (5182)	total: 6m 21s	remaining: 5m 52s
5300:	learn: 3.4860928	test: 3.6662882	best: 3.6662530 (5253)	total: 6m 29s	remaining: 5m 44s
5400:	learn: 3.4828367	test: 3.6662700	best: 3.6662484 (5321)	total: 6m 36s	remaining: 5m 37s
5500:	learn: 3.4797829	test: 3.6660975	best: 3.6660975 (5500)	total: 6m 43s	remaining: 5m 30s
5600:	learn: 3.4768231	test: 3.6660872	best: 3.6660678 (5587)	total: 6m 50s	remaining: 5m 22s
5700:	learn: 3.4739519	test: 3.6660508	best: 3.6660390 (5661)	total: 6m 58s	remaining: 5m 15s
5800:	learn: 3.4707940	test: 3.6659848	best: 3.6659343 (5792)	total: 7m 5s	remaining: 5m 8s
5900:	learn: 3.4677187	test: 3.6657393	best: 3.6657393 (5900)	total: 7m 13s	remaining: 5m 1s
6000:	learn: 3.4646996	test: 3.6657875	best: 3.6657221 (5930)	total: 7m 20s	remaining: 4m 53s
6100:	learn: 3.4619535	test: 3.6657326	best: 3.6656822 (6032)	t

In [79]:
# xgboost
current_random_state = 2985

X_train = df_train[features].values
X_test = df_test[features].values
y_train = df_train['target'].values

print('before impute X_train.shape is', X_train.shape, 'X_test.shape is', X_test.shape,
      'y_train.shape is ', y_train.shape)
print('y_train is ', y_train.mean())

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(np.vstack([X_train, X_test]))
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)

print('after impute X_train.shape is', X_train.shape, 'X_test.shape is', X_test.shape,
      'y_train.shape is ', y_train.shape)

xgb_params = {
    'n_estimators': 6000,
    'objective': 'reg:linear',
#     'objective': 'reg:tweedie',
#     'objective': 'reg:gamma',
    'eta': 0.008,
    'max_depth': 6,
    'nthread': 4,
#     'max_leaf_nodes': 101,
    'subsample': 0.60,
    'colsample_bytree': 0.55,
    'eval_metric': 'rmse',
    'lambda': 10,   
    'alpha': 40, 
    'silent': 1,
    'random_state': current_random_state
}

#     dtrain = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
#     dval = xgb.DMatrix(X_train[val_idx], y_train[val_idx])
#     dtest = xgb.DMatrix(X_test)
# xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)
# clf = lgb.train(xgb_params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
# # xgb_model = xgb.XGBRegressor()
# dtrain = xgb.DMatrix(X_train, y_train)
# dtest = xgb.DMatrix(X_test)
# xgb.train(X_train, y_train, early_stopping_rounds=10, eval_metric="rmse", eval_set=eval_set, verbose=100)
# xgb_model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)
# xgb_pred = xgb_model.predict(dtest)
# num_rounds = 1000
# xgb_model = xgb.train(xgb_params, trn_data, num_rounds, eval_sets = [dtrain, dval], 
#                       verbose_eval=100, early_stopping_rounds = 200)

oof_regressor_full_xgboost = np.zeros(len(df_train))
predictions_regressor_full_xgboost = np.zeros(len(df_test))
current_rmse_lst = []

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=current_random_state)
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    xgb_model = xgb.XGBRegressor(**xgb_params)
    xgb_model.fit(X_train[trn_idx], y_train[trn_idx], 
                  early_stopping_rounds=200, eval_metric="rmse", 
                  eval_set=[(X_train[trn_idx], y_train[trn_idx]),
                            (X_train[val_idx], y_train[val_idx])], 
                  verbose=20)
    oof_regressor_full_xgboost[val_idx] = xgb_model.predict(X_train[val_idx])
    predictions_regressor_full_catboost += xgb_model.predict(X_test) / folds.n_splits
    rmse_val = np.sqrt(mean_squared_error(y_train[val_idx], oof_regressor_full_xgboost[val_idx]))
    print('current rmse val is ', rmse_val)
    current_rmse_lst.append(rmse_val)

print('current_random_state: ', current_random_state)
print('current_rmse_lst xgboost: ', current_rmse_lst)
print('final rmse val xgboost: ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_full_xgboost)))

#######################################################################################################

# xgb_params = {
#     'n_estimators': 6000,
#     'objective': 'reg:linear',
#     'eta': 0.008,
#     'max_depth': 6,
#     'nthread': 4,
# #     'max_leaf_nodes': 101,
#     'subsample': 0.80,
#     'colsample_bytree': 0.7,
#     'eval_metric': 'rmse',
#     'lambda': 0.8,   
#     'alpha': 0.4, 
#     'silent': 1,
#     'random_state': current_random_state
# }

# current rmse val is  3.6587851036931576
# current_random_state:  2985
# current_rmse_lst xgboost:  [3.686406803015221, 3.648501745383582, 3.66815694794596, 3.671007538831685, 3.6587851036931576]
# final rmse val xgboost:  3.66659353116068

before impute X_train.shape is (201917, 113) X_test.shape is (123623, 113) y_train.shape is  (201917,)
y_train is  -0.3936
after impute X_train.shape is (201917, 113) X_test.shape is (123623, 113) y_train.shape is  (201917,)
fold n°0
[0]	validation_0-rmse:3.90491	validation_1-rmse:3.906
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 200 rounds.
[20]	validation_0-rmse:3.59608	validation_1-rmse:3.69817
[40]	validation_0-rmse:3.51925	validation_1-rmse:3.68851
[60]	validation_0-rmse:3.47432	validation_1-rmse:3.68869
[80]	validation_0-rmse:3.43649	validation_1-rmse:3.68968
[100]	validation_0-rmse:3.39863	validation_1-rmse:3.69356
[120]	validation_0-rmse:3.36756	validation_1-rmse:3.69956
[140]	validation_0-rmse:3.33886	validation_1-rmse:3.70097
[160]	validation_0-rmse:3.3022	validation_1-rmse:3.70688
[180]	validation_0-rmse:3.26862	validation_1-rmse:3.70973
[200]	validation_0-rmse:3.23368	val

In [47]:
# current_random_state = 1111
current_random_state = 3333

# def norm_df(df):
#     print('in norm_df')
#     df_norm = (df - df.mean()) / (df.max() - df.min())
#     return df_norm

# def fill_with_mean(df, feas):
#     print('in fill_with_mean')
#     for fea in feas:
#         df[fea].fillna(df[fea].mean(), inplace=True)
#     return df

X_train_brr = df_train[features].values
X_test_brr = df_test[features].values
y_train_brr = df_train['target'].values

print('before impute X_train.shape is', X_train_brr.shape, 'X_test.shape is', X_test_brr.shape,
      'y_train_brr.shape is ', y_train_brr.shape)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(np.vstack([X_train_brr, X_test_brr]))
X_train_brr = imp.transform(X_train_brr)
X_test_brr = imp.transform(X_test_brr)

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(np.vstack([X_train_brr, X_test_brr]))
X_train_brr = scaler.transform(X_train_brr)
X_test_brr = scaler.transform(X_test_brr)

oof_regressor_full_brr = np.zeros(len(df_train))
predictions_regressor_full_brr = np.zeros(len(df_test))

print('after impute X_train.shape is', X_train_brr.shape, 'X_test.shape is', X_test_brr.shape,
      'y_train_brr.shape is ', y_train_brr.shape)

print('start bayes ridge regression training')


def compute_CV_brr(tol, alpha_1, alpha_2, lambda_1, lambda_2, normalize):
    print('###################################################')
    print('tol:{}, alpha_1:{}, alpha_2:{}, lambda_1:{}, lambda_2:{}, normalize:{}'.format(
        tol, alpha_1, alpha_2, lambda_1, lambda_2, normalize))
    
    current_rmse_lst = [] 
    oof_regressor_full_brr = np.zeros(len(df_train))
    predictions_regressor_full_brr = np.zeros(len(df_test))

    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=current_random_state)
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train_brr, df_train['outliers'].values)):
    #     print("fold n°{}".format(fold_))
        brr = BayesianRidge(n_iter=1000, tol=1e-7, alpha_1=1e-03, alpha_2=1e-04, normalize=True, verbose=True) 
        brr.fit(X_train_brr[trn_idx], y_train_brr[trn_idx])
        oof_regressor_full_brr[val_idx] = brr.predict(X_train_brr[val_idx])
        predictions_regressor_full_brr += brr.predict(X_test_brr) / folds.n_splits
        rmse_val = np.sqrt(mean_squared_error(y_train_brr[val_idx], oof_regressor_full_brr[val_idx]))
    #     print('current rmse val brr is ', rmse_val)
        current_rmse_lst.append(rmse_val)

    final_rmse_val_brr = np.sqrt(mean_squared_error(y_train_brr, oof_regressor_full_brr))
    
    print('current_random_state: ', current_random_state)
    print('current_rmse_lst brr: ', current_rmse_lst)
    print('final brr rmse: ', final_rmse_val_brr)
    
    return current_rmse_lst, final_rmse_val_brr

# tol_lst = [0.001, 0.01, 0.1]
tol_lst = [0.01, 0.1]
alpha_1_lst = [1e-06, 1e-04, 1e-01]
alpha_2_lst = [1e-06]
lambda_1_lst = [1e-06, 1e-04, 1e-01]
lambda_2_lst = [1e-06, 1e-04]
normalize_lst = [False]

for tol, alpha_1, alpha_2, lambda_1, lambda_2, normalize in itertools.product(tol_lst, 
            alpha_1_lst, alpha_2_lst, lambda_1_lst, lambda_2_lst, normalize_lst):
    compute_CV_brr(tol, alpha_1, alpha_2, lambda_1, lambda_2, normalize)
    

# stack_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
# stack_outcome_df['target'] = predictions_regressor_full_brr
# final_rmse_val_brr = round(final_rmse_val_brr, 9)
# stack_outcome_df.to_csv(file_path + '/submission/brr_' + str(final_rmse_val_brr) + '.csv', index=False)

# current_random_state:  3333
# current_rmse_lst brr:  [3.878985619489133, 3.806160738865977, 624.0305245668886, 3.7817687274687835, 3.8008537147485706]
# final brr rmse:  279.09443571234505

# current_random_state:  3333   tol=0.001
# current_rmse_lst brr:  [3.759385152413941, 3.761745139549289, 35.77986635675397, 3.7549984561083884, 3.7730677021533547]
# final brr rmse:  16.351183762209807

before impute X_train.shape is (201917, 113) X_test.shape is (123623, 113) y_train_brr.shape is  (201917,)
after impute X_train.shape is (201917, 113) X_test.shape is (123623, 113) y_train_brr.shape is  (201917,)
start bayes ridge regression training
###################################################
tol:0.01, alpha_1:1e-06, alpha_2:1e-06, lambda_1:1e-06, lambda_2:1e-06, normalize:False
Convergence after  18  iterations
Convergence after  13  iterations
Convergence after  14  iterations
Convergence after  15  iterations
Convergence after  17  iterations
current_random_state:  3333
current_rmse_lst brr:  [3.759385151499871, 3.7617451398692108, 35.77967355254788, 3.7549984562381513, 3.7730677024836043]
final brr rmse:  16.35109938373042
###################################################
tol:0.01, alpha_1:1e-06, alpha_2:1e-06, lambda_1:1e-06, lambda_2:0.0001, normalize:False
Convergence after  18  iterations
Convergence after  13  iterations
Convergence after  14  iterations
Convergence

Convergence after  18  iterations
Convergence after  13  iterations
Convergence after  14  iterations
Convergence after  15  iterations
Convergence after  17  iterations
current_random_state:  3333
current_rmse_lst brr:  [3.759385151499871, 3.7617451398692108, 35.77967355254788, 3.7549984562381513, 3.7730677024836043]
final brr rmse:  16.35109938373042
###################################################
tol:0.01, alpha_1:0.1, alpha_2:1e-06, lambda_1:0.1, lambda_2:0.0001, normalize:False
Convergence after  18  iterations
Convergence after  13  iterations
Convergence after  14  iterations
Convergence after  15  iterations
Convergence after  17  iterations
current_random_state:  3333
current_rmse_lst brr:  [3.759385151499871, 3.7617451398692108, 35.77967355254788, 3.7549984562381513, 3.7730677024836043]
final brr rmse:  16.35109938373042
###################################################
tol:0.1, alpha_1:1e-06, alpha_2:1e-06, lambda_1:1e-06, lambda_2:1e-06, normalize:False
Convergence af

Convergence after  18  iterations
Convergence after  13  iterations
Convergence after  14  iterations
Convergence after  15  iterations
Convergence after  17  iterations
current_random_state:  3333
current_rmse_lst brr:  [3.759385151499871, 3.7617451398692108, 35.77967355254788, 3.7549984562381513, 3.7730677024836043]
final brr rmse:  16.35109938373042
###################################################
tol:0.1, alpha_1:0.1, alpha_2:1e-06, lambda_1:0.1, lambda_2:1e-06, normalize:False
Convergence after  18  iterations
Convergence after  13  iterations
Convergence after  14  iterations
Convergence after  15  iterations
Convergence after  17  iterations
current_random_state:  3333
current_rmse_lst brr:  [3.759385151499871, 3.7617451398692108, 35.77967355254788, 3.7549984562381513, 3.7730677024836043]
final brr rmse:  16.35109938373042
###################################################
tol:0.1, alpha_1:0.1, alpha_2:1e-06, lambda_1:0.1, lambda_2:0.0001, normalize:False
Convergence after  

In [ ]:
current_random_state = 1111

X_train = df_train[features].values
X_test = df_test[features].values
y_train = df_train['target'].values

print('before impute X_train.shape is', X_train.shape, 'X_test.shape is', X_test.shape,
      'y_train.shape is ', y_train.shape)
print('y_train is ', y_train.mean())

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(np.vstack([X_train, X_test]))
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(np.vstack([X_train, X_test]))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

oof_regressor_full_rf = np.zeros(len(df_train))
predictions_regressor_full_rf = np.zeros(len(df_test))

current_rmse_lst = []

print('after impute X_train.shape is', X_train.shape, 'X_test.shape is', X_test.shape,
      'y_train.shape is ', y_train.shape)

print('start bayes ridge regression training')
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=current_random_state)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train_brr, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    
    rfr = RandomForestRegressor(n_estimators=2000, max_depth=7, min_samples_split=51, 
                               min_samples_leaf=31, max_features=53, 
                               random_state=current_random_state)
    print('start random forest regressor training...')
    clf = rfr.fit(X_train[trn_idx], y_train[trn_idx])
    oof_regressor_full_rf[val_idx] = rfr.predict(X_train[val_idx])    
    predictions_regressor_full_rf += rfr.predict(X_test) / folds.n_splits

    rmse_val = np.sqrt(mean_squared_error(y_train[val_idx], oof_regressor_full_rf[val_idx]))
    print('current rmse val brr is ', rmse_val)
    current_rmse_lst.append(rmse_val)

final_rmse_val_brr = np.sqrt(mean_squared_error(y_train_brr, oof_regressor_full_lgb_debug))

print('current_random_state: ', current_random_state)
print('current_rmse_lst brr: ', current_rmse_lst)
print('final brr rmse: ', final_rmse_val_brr)

stack_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
stack_outcome_df['target'] = predictions_regressor_full_lgb_debug
final_rmse_val_brr = round(final_rmse_val_brr, 9)
stack_outcome_df.to_csv(file_path + '/submission/rfr_' + str(final_rmse_val_brr) + '.csv', index=False)


In [54]:
from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=600, n_jobs=4, min_samples_split=61, 
                          max_depth=8, min_samples_leaf=37, max_features=53, 
                          verbose=100, random_state=1234)

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

oof_regressor_full_6 = np.zeros(len(df_train))
predictions_regressor_full_6 = np.zeros(len(df_test))

df_train_etr = df_train.copy().fillna(0.0)
df_test_etr = df_test.copy().fillna(0.0)

rmse_val_lst = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    X_train_etr = df_train_etr.iloc[trn_idx][features].values.copy()
    y_train_etr = df_train_etr['target'].iloc[trn_idx].values.copy()
    X_val_etr = df_train_etr.iloc[val_idx][features].values.copy()
    y_val_etr = df_train_etr['target'].iloc[val_idx].values.copy()
    
#     etr.fit(X_train_etr, y_train_etr)
    etr.fit(df_train_etr.iloc[trn_idx][features], df_train_etr['target'].iloc[trn_idx])
    oof_regressor_full_6[val_idx] = etr.predict(df_train_etr.iloc[val_idx][features].values)
#                                                            num_iteration=catboost_model.best_iteration_)   
    
    predictions_regressor_full_6 += etr.predict(df_test_etr[features].values) / folds.n_splits
    rmse_val = np.sqrt(mean_squared_error(df_train_etr.iloc[val_idx]['target'].values, 
                                          oof_regressor_full_6[val_idx]))
    print('current rmse val is ', rmse_val)
    rmse_val_lst.append(rmse_val)
    
# current rmse val is  3.694068689339914

print('rmse_val_lst is ', rmse_val_lst)
print('final rmse val 4 is ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_full_6)))

# rmse_val_lst is  [3.6843289467801155, 3.694446552143381, 3.7123359542336245, 3.6994698300129354, 3.6899055614700464]
# final rmse val 4 is  3.6961096025459783

fold n°0
building tree 1 of 600
building tree 2 of 600
building tree 3 of 600
building tree 4 of 600
building tree 5 of 600[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.8s

building tree 6 of 600[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.8s

building tree 7 of 600[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    0.8s

building tree 8 of 600[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.8s

building tree 9 of 600[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.7s

building tree 10 of 600[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    1.7s

building tree 11 of 600[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    1.7s

building tree 12 of 600[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    1.8s

building tree 13 of 600[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    2.6s

building tree 14 of 600[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.6s

building tree 15 of 600[Parallel(n_jobs=4)]: Done  11 

building tree 101 of 600[Parallel(n_jobs=4)]: Done  97 tasks      | elapsed:   21.4s

building tree 102 of 600[Parallel(n_jobs=4)]: Done  98 tasks      | elapsed:   21.4s

building tree 103 of 600[Parallel(n_jobs=4)]: Done  99 tasks      | elapsed:   21.5s

building tree 104 of 600[Parallel(n_jobs=4)]: Done 100 tasks      | elapsed:   21.6s

building tree 105 of 600[Parallel(n_jobs=4)]: Done 101 tasks      | elapsed:   22.3s

building tree 106 of 600[Parallel(n_jobs=4)]: Done 102 tasks      | elapsed:   22.3s

building tree 107 of 600[Parallel(n_jobs=4)]: Done 103 tasks      | elapsed:   22.4s

building tree 108 of 600[Parallel(n_jobs=4)]: Done 104 tasks      | elapsed:   22.4s

building tree 109 of 600[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:   23.2s

building tree 110 of 600[Parallel(n_jobs=4)]: Done 106 tasks      | elapsed:   23.2s

building tree 111 of 600[Parallel(n_jobs=4)]: Done 107 tasks      | elapsed:   23.3s

building tree 112 of 600[Parallel(n_jobs=4)]: Done 108

building tree 197 of 600[Parallel(n_jobs=4)]: Done 193 tasks      | elapsed:   41.3s

building tree 198 of 600[Parallel(n_jobs=4)]: Done 194 tasks      | elapsed:   41.4s

building tree 199 of 600[Parallel(n_jobs=4)]: Done 195 tasks      | elapsed:   41.6s

building tree 200 of 600[Parallel(n_jobs=4)]: Done 196 tasks      | elapsed:   41.7s

building tree 201 of 600[Parallel(n_jobs=4)]: Done 197 tasks      | elapsed:   42.1s

building tree 202 of 600[Parallel(n_jobs=4)]: Done 198 tasks      | elapsed:   42.3s

building tree 203 of 600[Parallel(n_jobs=4)]: Done 199 tasks      | elapsed:   42.4s

building tree 204 of 600[Parallel(n_jobs=4)]: Done 200 tasks      | elapsed:   42.5s

building tree 205 of 600[Parallel(n_jobs=4)]: Done 201 tasks      | elapsed:   42.9s

building tree 206 of 600[Parallel(n_jobs=4)]: Done 202 tasks      | elapsed:   43.1s

building tree 207 of 600[Parallel(n_jobs=4)]: Done 203 tasks      | elapsed:   43.2s

building tree 208 of 600[Parallel(n_jobs=4)]: Done 204

building tree 293 of 600[Parallel(n_jobs=4)]: Done 289 tasks      | elapsed:  1.0min

building tree 294 of 600[Parallel(n_jobs=4)]: Done 290 tasks      | elapsed:  1.0min

building tree 295 of 600[Parallel(n_jobs=4)]: Done 291 tasks      | elapsed:  1.0min

building tree 296 of 600[Parallel(n_jobs=4)]: Done 292 tasks      | elapsed:  1.0min

building tree 297 of 600[Parallel(n_jobs=4)]: Done 293 tasks      | elapsed:  1.1min

building tree 298 of 600[Parallel(n_jobs=4)]: Done 294 tasks      | elapsed:  1.1min

building tree 299 of 600[Parallel(n_jobs=4)]: Done 295 tasks      | elapsed:  1.1min

building tree 300 of 600[Parallel(n_jobs=4)]: Done 296 tasks      | elapsed:  1.1min

building tree 301 of 600[Parallel(n_jobs=4)]: Done 297 tasks      | elapsed:  1.1min

building tree 302 of 600[Parallel(n_jobs=4)]: Done 298 tasks      | elapsed:  1.1min

building tree 303 of 600[Parallel(n_jobs=4)]: Done 299 tasks      | elapsed:  1.1min

building tree 304 of 600[Parallel(n_jobs=4)]: Done 300

building tree 389 of 600[Parallel(n_jobs=4)]: Done 385 tasks      | elapsed:  1.4min

building tree 390 of 600[Parallel(n_jobs=4)]: Done 386 tasks      | elapsed:  1.4min

building tree 391 of 600[Parallel(n_jobs=4)]: Done 387 tasks      | elapsed:  1.4min

building tree 392 of 600[Parallel(n_jobs=4)]: Done 388 tasks      | elapsed:  1.4min

building tree 393 of 600[Parallel(n_jobs=4)]: Done 389 tasks      | elapsed:  1.4min

building tree 394 of 600[Parallel(n_jobs=4)]: Done 390 tasks      | elapsed:  1.4min

building tree 395 of 600[Parallel(n_jobs=4)]: Done 391 tasks      | elapsed:  1.4min

building tree 396 of 600[Parallel(n_jobs=4)]: Done 392 tasks      | elapsed:  1.4min

building tree 397 of 600[Parallel(n_jobs=4)]: Done 393 tasks      | elapsed:  1.4min

building tree 398 of 600[Parallel(n_jobs=4)]: Done 394 tasks      | elapsed:  1.4min

building tree 399 of 600[Parallel(n_jobs=4)]: Done 395 tasks      | elapsed:  1.4min

building tree 400 of 600[Parallel(n_jobs=4)]: Done 396

...

[Parallel(n_jobs=4)]: Done 472 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 473 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 474 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 475 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 476 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 477 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 478 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 479 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 480 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 481 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 482 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 483 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 484 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 485 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 486 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 487 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done  52 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  54 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  55 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  57 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  58 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  59 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  60 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  61 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  62 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  63 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  65 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  66 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  67 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 215 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 216 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 217 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 218 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 219 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 220 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 221 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 222 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 223 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 224 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 225 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 226 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 227 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 228 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 229 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 230 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 396 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 397 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 398 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 399 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 400 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 401 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 402 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 403 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 404 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 405 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 406 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 407 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 408 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 409 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 410 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 411 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 567 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 568 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 569 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 570 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 571 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 572 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 573 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 574 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 575 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 576 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 577 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 578 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 579 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 580 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 581 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 582 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Do

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

# etr = ExtraTreeRegressor(criterion=’mse’, splitter=’random’, max_depth=7, 
#                    min_samples_split=7, min_samples_leaf=31, min_weight_fraction_leaf=0.0, 
#                    random_state=1234, min_impurity_decrease=0.0, min_impurity_split=None, max_leaf_nodes=None)


etr = ExtraTreesRegressor(n_estimators=600, n_jobs=4, min_samples_split=61, 
                          max_depth=8, min_samples_leaf=37, max_features=53, 
                          verbose=100, random_state=1234)

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

oof_regressor_full_6 = np.zeros(len(df_train))
predictions_regressor_full_6 = np.zeros(len(df_test))

df_train_etr = df_train.copy().fillna(0.0)
df_test_etr = df_test.copy().fillna(0.0)

rmse_val_lst = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    X_train_etr = df_train_etr.iloc[trn_idx][features].values.copy()
    y_train_etr = df_train_etr['target'].iloc[trn_idx].values.copy()
    X_val_etr = df_train_etr.iloc[val_idx][features].values.copy()
    y_val_etr = df_train_etr['target'].iloc[val_idx].values.copy()
    
#     etr.fit(X_train_etr, y_train_etr)
    etr.fit(df_train_etr.iloc[trn_idx][features], df_train_etr['target'].iloc[trn_idx])
    oof_regressor_full_6[val_idx] = etr.predict(df_train_etr.iloc[val_idx][features].values)
#                                                            num_iteration=catboost_model.best_iteration_)   
    
    predictions_regressor_full_6 += etr.predict(df_test_etr[features].values) / folds.n_splits
    rmse_val = np.sqrt(mean_squared_error(df_train_etr.iloc[val_idx]['target'].values, 
                                          oof_regressor_full_6[val_idx]))
    print('current rmse val is ', rmse_val)
    rmse_val_lst.append(rmse_val)
    
# current rmse val is  3.694068689339914

print('rmse_val_lst is ', rmse_val_lst)
print('final rmse val 4 is ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_full_6)))

In [66]:
from sklearn.linear_model import BayesianRidge

oof_lst = [oof_regressor_full_1, oof_regressor_full_2, 
           oof_regressor_full_3, oof_regressor_full_4, 
#            oof_regressor_full_6
]
train_stack = np.vstack(oof_lst).transpose()

predictions_lst = [predictions_regressor_full_1, predictions_regressor_full_2, 
                   predictions_regressor_full_3, predictions_regressor_full_4, 
#                    predictions_regressor_full_6
]
test_stack = np.vstack(predictions_lst).transpose()

rmse_one_lst = []
for oof in oof_lst:
    rmse_one = np.sqrt(mean_squared_error(df_train['target'].values, oof))
    rmse_one_lst.append(rmse_one)
print('rmse_one_lst is ', rmse_one_lst)

folds_stack = RepeatedKFold(n_splits=5, n_repeats=1, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions_stack = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack, df_train['target'].values)):
#     print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], df_train['target'].iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], df_train['target'].iloc[val_idx].values
    
    brr = BayesianRidge()
    brr.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = brr.predict(val_data)
    predictions_stack += brr.predict(test_stack) / 5
    
rmse_stack = np.sqrt(mean_squared_error(df_train['target'].values, oof_stack))
print('rmse_stack is ', rmse_stack)

stack_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
stack_outcome_df['target'] = predictions_stack
rmse_stack = round(rmse_stack, 9)
stack_outcome_df.to_csv(file_path + '/submission/stack_' + str(rmse_stack) + '.csv', index=False)

# 2个stacking rmse_stack is  3.655309663680292
# 3个stacking rmse_stack is  3.6533463122516023
# 4个stacking rmse_stack is  3.651213275367461
# 5个stacking rmse_stack is  3.6511967869076782

# random_state=4590
# rmse_one_lst is  [3.6579402971548394, 3.6582454648471354, 3.6611204491816722, 3.6571019611013256, 3.6961096025459783]
# rmse_stack is  3.6511967869076782   



rmse_one_lst is  [3.6579402971548394, 3.6582454648471354, 3.6611204491816722, 3.6571019611013256]
rmse_stack is  3.651213275367461


In [ ]:
target_col = 'target'

final_outcome_df['target']

plt.figure(figsize=(8,6))
plt.scatter(range(final_outcome_df.shape[0]), np.sort(final_outcome_df['target'].values))
plt.title("value distrubition of Loyalty score")
plt.xlabel('index', fontsize=12)
plt.ylabel('Loyalty Score', fontsize=12)
plt.show()

In [ ]:
final_outcome_df['target'].plot.hist(bins=50, alpha=0.5)
final_outcome_df['target'].mean()

In [ ]:
target_col = 'target'

final_outcome_df['target']

plt.figure(figsize=(8,6))
plt.scatter(range(df_train.shape[0]), np.sort(df_train['target'].values))
plt.title("value distrubition of Loyalty score")
plt.xlabel('index', fontsize=12)
plt.ylabel('Loyalty Score', fontsize=12)
plt.show()

In [ ]:
rmse_lst, ratio_lst = [], []

for i in range(1, 100):
    ratio = i*0.01
    if ratio<=0.6:
        print('ratio is ', ratio)
        rmse_val = compute_rmse_check_ratio(df_train, trn_idx_list, val_idx_list, preds_classifier_list, 
                                 preds_regressor_full_list, preds_regressor_partial_list, 
                                 ratio)
        ratio_lst.append(ratio)
        rmse_lst.append(rmse_val)

plt.plot(ratio_lst, rmse_lst, linewidth=1.5)
plt.show()

In [ ]:
# plt.plot(ratio_lst, rmse_lst, linewidth=1.5)
# plt.show()

min(rmse_lst)

In [ ]:
final_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
final_outcome_df['target'] = get_final_results(df_test, predictions_classifier, 
                                               predictions_regressor_full, 
                                               predictions_regressor_partial,
                                               partial_ratio=0.25)
final_outcome_df.to_csv(file_path + '/submission/submission_first_kernel_tsg.csv', index=False)

In [ ]:
param_classifier = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
#          'max_depth': 6,
         'max_depth': -1,
         'learning_rate': 0.01,
#          "boosting": "rf",
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
#          "random_state": 2333,
         'random_state': RANDOM_STATE_TSG
}

param_regressor = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.007,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
#          "random_state": 2333,
         'random_state': RANDOM_STATE_TSG
}

# param_regressor = {
#          'num_leaves': 31,
#          'min_data_in_leaf': 30, 
#          'objective':'regression',
#          'max_depth': -1,
#          'learning_rate': 0.008,
#          "boosting": "gbdt",
#          "feature_fraction": 0.5,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.5 ,
#          "bagging_seed": 11,
#          "metric": 'rmse',
#          "lambda_l1": 0.1,
#          "verbosity": -1,
#          "random_state": 2333
# }

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE_TSG)
# folds = KFold(n_splits=5, shuffle=True, random_state=15)

oof_classifier = np.zeros(len(df_train))
oof_regressor_full = np.zeros(len(df_train))
oof_regressor_partial = np.zeros(len(df_train))
oof_regressor_merged = np.zeros(len(df_train))

predictions_classifier = np.zeros(len(df_test))
predictions_regressor_partial = np.zeros(len(df_test))
predictions_regressor_full = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

target = None

trn_idx_list, val_idx_list, preds_classifier_list, preds_regressor_full_list, preds_regressor_partial_list = [], [], [], [], []

start = time.time()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    
    # 训练一个分类器 决定是否为 outlier
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['outliers'].iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['outliers'].iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 10000
    print('start classifier training...')
    clf = lgb.train(param_classifier, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof_classifier[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

    auc = roc_auc_score(df_train['outliers'].iloc[val_idx], oof_classifier[val_idx])
    print('current auc is ', auc)
    
    predictions_classifier += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = features
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold_ + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    # 训练一个Regressor 决定target值为多少
    
    # full regressor 
    trn_idx_new = trn_idx
    val_idx_new = val_idx

    trn_data = lgb.Dataset(df_train.iloc[trn_idx_new][features], label=df_train['target'].iloc[trn_idx_new], 
                           categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx_new][features], label=df_train['target'].iloc[val_idx_new], 
                           categorical_feature=categorical_feats)

    num_round = 10000
    print('start regressor training...')
    clf = lgb.train(param_regressor, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_regressor_full[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)    
    predictions_regressor_full += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
    
    # partial regressor
    trn_idx_new = trn_idx[df_train['outliers'].iloc[trn_idx]==0]
    print('trn_idx.shape is ', trn_idx.shape, 'trn_idx_new.shape is ', trn_idx_new.shape)
    val_idx_new = val_idx[df_train['outliers'].iloc[val_idx]==0]
    print('val_idx.shape is ', val_idx.shape, 'val_idx_new.shape is ', val_idx_new.shape)

    trn_data = lgb.Dataset(df_train.iloc[trn_idx_new][features], label=df_train['target'].iloc[trn_idx_new], 
                           categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx_new][features], label=df_train['target'].iloc[val_idx_new], 
                           categorical_feature=categorical_feats)

    num_round = 10000
    print('start regressor training...')
    clf = lgb.train(param_regressor, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_regressor_partial[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions_regressor_partial += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
    oof_regressor_merged[val_idx], rmse_val = compute_val_rmse(df_train, trn_idx, 
                                                               val_idx, oof_classifier[val_idx], 
                                                               oof_regressor_full[val_idx], 
                                                               oof_regressor_partial[val_idx])
    print('current rmse val is ', rmse_val)
    
    trn_idx_list.append(trn_idx)
    val_idx_list.append(val_idx)
    preds_classifier_list.append(oof_classifier[val_idx])
    preds_regressor_full_list.append(oof_regressor_full[val_idx])
    preds_regressor_partial_list.append(oof_regressor_partial[val_idx])
    
print('final rmse val is ', np.sqrt(mean_squared_error(df_train['target'].values, oof_regressor_merged)))
print('final auc is ', roc_auc_score(df_train['outliers'].values, oof_classifier))

# CV score: 0.04615 
# final auc is  0.8738057712001387

# predictions_is_outlier = pd.DataFrame({"card_id": df_test["card_id"].values})
# outlier_num = int(df_train['outliers'].mean()*df_test.shape[0])  # outlier 个数
outlier_num = 10
# predictions_is_outlier['target'] =  predictions_classifier
# outlier_id = list(predictions_is_outlier.sort_values(by='target',ascending = False).head(outlier_num)['card_id'])

final_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
final_outcome_df['target'] = get_final_results(df_test, predictions_classifier, 
                                               predictions_regressor_full, predictions_regressor_partial,
                                               partial_ratio=0.25)

final_outcome_df.to_csv(file_path + '/submission/submission_first_kernel_tsg.csv', index=False)

In [ ]:
# def compute_val_rmse(df_train, trn_idx, val_idx, preds_classifier, preds_regressor):
#     print('in compute_val_rmse')
#     df_is_outlier = pd.DataFrame({"card_id": df_train["card_id"].iloc[val_idx].values})
#     df_is_outlier['outlier'] =  preds_classifier
#     outlier_num = int(df_train['outliers'].iloc[trn_idx].mean()*len(val_idx))  # outlier 个数
#     outlier_num = int(0.2*outlier_num) # 只取一半  
#     outlier_num = 10
#     # 0.5  4.16
#     # 0.2  3.85
#     # 10  
#     outlier_id = list(df_is_outlier.sort_values(by='outlier',ascending = False).head(outlier_num)['card_id'])

#     final_outcome_df = pd.DataFrame({"card_id": df_train["card_id"].iloc[val_idx].values})
#     final_outcome_df['target'] = preds_regressor
#     final_outcome_df.loc[final_outcome_df['card_id'].isin(outlier_id), 'target'] = -33.21928095
#     final_rmse = np.sqrt(mean_squared_error(df_train['target'].iloc[val_idx], 
#                                             final_outcome_df['target'].values))    
#     return final_rmse    


# RANDOM_STATE_TSG = 10007
# # target = df_train['outliers']

# features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers', 'target']]
# features_old = features
# features = list(df_train[features].select_dtypes(include=['number', 'bool']).columns.values)  # 去除非数值、非布尔类型数据
# # categorical_feats = [c for c in features if 'feature_' in c]

# categorical_feats = []
# print(len(features), len(categorical_feats))

# param_classifier = {
#          'num_leaves': 31,
#          'min_data_in_leaf': 30, 
#          'objective':'binary',
#          'max_depth': 6,
#          'learning_rate': 0.01,
#          "boosting": "rf",
#          "feature_fraction": 0.9,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.9 ,
#          "bagging_seed": 11,
#          "metric": 'binary_logloss',
#          "lambda_l1": 0.1,
#          "verbosity": -1,
#          "random_state": 2333
# }

# param_regressor = {
#          'num_leaves': 31,
#          'min_data_in_leaf': 30, 
#          'objective':'regression',
#          'max_depth': -1,
#          'learning_rate': 0.007,
#          "boosting": "gbdt",
#          "feature_fraction": 0.9,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.9 ,
#          "bagging_seed": 11,
#          "metric": 'rmse',
#          "lambda_l1": 0.1,
#          "verbosity": -1,
#          "random_state": 2333
# }

# folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE_TSG)
# # folds = KFold(n_splits=5, shuffle=True, random_state=15)

# oof_classifier = np.zeros(len(df_train))
# oof_regressor = np.zeros(len(df_train))

# predictions_classifier = np.zeros(len(df_test))
# predictions_regressor = np.zeros(len(df_test))
# feature_importance_df = pd.DataFrame()

# start = time.time()

# for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
#     print("fold n°{}".format(fold_))
    
#     # 训练一个分类器 决定是否为 outlier
#     trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['outliers'].iloc[trn_idx], categorical_feature=categorical_feats)
#     val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['outliers'].iloc[val_idx], categorical_feature=categorical_feats)

#     num_round = 10000
#     print('start classifier training...')
#     clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
#     oof_classifier[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

#     auc = roc_auc_score(target.iloc[val_idx], oof[val_idx])
#     print('auc is ', auc)
    
#     predictions_classifier += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
# #     fold_importance_df = pd.DataFrame()
# #     fold_importance_df["feature"] = features
# #     fold_importance_df["importance"] = clf.feature_importance()
# #     fold_importance_df["fold"] = fold_ + 1
# #     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
#     # 训练一个Regressor 决定target值为多少
# #     trn_idx_new = trn_idx[df_train['outliers'].iloc[trn_idx]==0]
# #     print('trn_idx.shape is ', trn_idx.shape, 'trn_idx_new.shape is ', trn_idx_new.shape)
# #     val_idx_new = val_idx[df_train['outliers'].iloc[val_idx]==0]
# #     print('val_idx.shape is ', val_idx.shape, 'val_idx_new.shape is ', val_idx_new.shape)
    
#     trn_idx_new = trn_idx
#     val_idx_new = val_idx

#     trn_data = lgb.Dataset(df_train.iloc[trn_idx_new][features], label=df_train['target'].iloc[trn_idx_new], 
#                            categorical_feature=categorical_feats)
#     val_data = lgb.Dataset(df_train.iloc[val_idx_new][features], label=df_train['target'].iloc[val_idx_new], 
#                            categorical_feature=categorical_feats)

#     num_round = 10000
#     print('start regressor training...')
#     clf = lgb.train(param_regressor, trn_data, num_round, valid_sets = [trn_data, val_data], 
#                     verbose_eval=100, early_stopping_rounds = 200)
#     oof_regressor[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
#     predictions_regressor += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
#     final_rmse = compute_val_rmse(df_train, trn_idx, val_idx, oof_classifier[val_idx], oof_regressor[val_idx])
#     print('final_rmse is ', final_rmse)

# print("CV score: {:<8.5f}".format(log_loss(target, oof)))
# print('final auc is ', roc_auc_score(target, oof))

# # CV score: 0.04615 
# # final auc is  0.8738057712001387

# predictions_is_outlier = pd.DataFrame({"card_id": df_test["card_id"].values})
# # outlier_num = int(df_train['outliers'].mean()*df_test.shape[0])  # outlier 个数
# outlier_num = 10
# predictions_is_outlier['target'] =  predictions_classifier
# outlier_id = list(predictions_is_outlier.sort_values(by='target',ascending = False).head(outlier_num)['card_id'])

# final_outcome_df = pd.DataFrame({"card_id": df_test["card_id"].values})
# final_outcome_df['target'] = predictions_regressor

# print('prev final_outcome_df[target].sum() ', final_outcome_df['target'].sum())
# final_outcome_df.loc[final_outcome_df['card_id'].isin(outlier_id), 'target'] = -33.21928095
# print('after final_outcome_df[target].sum() ', final_outcome_df['target'].sum())

# final_outcome_df.to_csv(file_path + '/submission/submission_first_kernel_tsg.csv', index=False)

In [ ]:
# RANDOM_STATE_TSG = 10007
# target = df_train['outliers']

# features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers', 'target']]
# features = list(df_train[features].select_dtypes(include=['number', 'bool']).columns.values)  # 去除非数值、非布尔类型数据
# # categorical_feats = [c for c in features if 'feature_' in c]

# categorical_feats = []
# print(len(features), len(categorical_feats))

# param_classifier = {
#          'num_leaves': 31,
#          'min_data_in_leaf': 30, 
#          'objective':'binary',
#          'max_depth': 6,
#          'learning_rate': 0.01,
#          "boosting": "rf",
#          "feature_fraction": 0.9,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.9 ,
#          "bagging_seed": 11,
#          "metric": 'binary_logloss',
#          "lambda_l1": 0.1,
#          "verbosity": -1,
#          "random_state": 2333
# }

# folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE_TSG)
# # folds = KFold(n_splits=5, shuffle=True, random_state=15)
# oof = np.zeros(len(df_train))
# predictions = np.zeros(len(df_test))
# feature_importance_df = pd.DataFrame()

# start = time.time()

# for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
#     print("fold n°{}".format(fold_))
    
#     # 训练一个分类器 决定是否为 outlier
#     trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['outliers'].iloc[trn_idx], categorical_feature=categorical_feats)
#     val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['outliers'].iloc[val_idx], categorical_feature=categorical_feats)

#     num_round = 10000
#     clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
#     oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

#     auc = roc_auc_score(target.iloc[val_idx], oof[val_idx])
#     print('auc is ', auc)
    
# #     fold_importance_df = pd.DataFrame()
# #     fold_importance_df["feature"] = features
# #     fold_importance_df["importance"] = clf.feature_importance()
# #     fold_importance_df["fold"] = fold_ + 1
# #     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
#     predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

# print("CV score: {:<8.5f}".format(log_loss(target, oof)))
# print('final auc is ', roc_auc_score(target, oof))

# # CV score: 0.04615 
# # final auc is  0.8738057712001387

# predictions_is_outlier = pd.DataFrame({"card_id": df_test["card_id"].values})
# outlier_num = int(df_train['outliers'].mean()*df_test.shape[0])  # outlier 个数
# print('outlier_num is ', outlier_num)
# predictions_is_outlier['target'] =  predictions

# outlier_id = list(predictions_is_outlier.sort_values(by='target',ascending = False).head(outlier_num)['card_id'])
# print('len of outlier_id is ', len(outlier_id), 'first 20 of outlier_id is ', outlier_id[:20])

In [ ]:
# 想法：对于有outlier倾向的 可以用全体做样本训练的得到的值
# 想法：对于没有outlier倾向的 可以只使用部分样本得到的值